In [3]:
import cv2
import numpy as np
import random
import os

In [3]:
from aug import apply_defect

(0.9029411764705882, 0.7752941176470588, 0.023529411764705882, 0.023529411764705882)


In [18]:
# Функция для сохранения маски
def save_mask(mask, folder, image_name):
    # Указываем путь для сохранения
    mask_filename = os.path.join(folder, f"MASK.{image_name}")
    cv2.imwrite(mask_filename, mask)

In [28]:
image_folder = 'res\\images\\back'
output_folder = 'res\\images\\output'

In [130]:
defects_path = 'res\\defects'
defect1 = 'badlock'
defect2 = 'missing_screw'

# Пути к дефектам
defects_path1 = os.path.join(defects_path, defect1)
defects_list1 = os.listdir(defects_path1)
defects_path2 = os.path.join(defects_path, defect2)
defects_list2 = os.listdir(defects_path2)

defects_list = [(defects_path1, defects_list1, 1), (defects_path2, defects_list2, 0)]  # Список всех дефектов

# ¯\_(ツ)_/¯ Возможно избыточный код, но ладно


N = 6  # Задаем максимальное количество дефектов

for image_name in os.listdir(image_folder):
    if not image_name.lower().endswith(('.jpg', '.jpeg', '.webp')):
        continue
    
    # Загружаем изображение
    
    
    for i in range(15):

        image_path = os.path.join(image_folder, image_name)
        laptop_img = cv2.imread(image_path)
        mask_name = os.path.join(image_folder, 'masks', f'MASK.{image_name}')
        mask_img = cv2.imread(mask_name, 0)
            # Приводим маску к размеру изображения
        mask_resized = cv2.resize(mask_img, (laptop_img.shape[1], laptop_img.shape[0]))

        mask_resized = eroded_mask = cv2.erode(mask_resized, np.ones((5, 5), np.uint8), iterations=1)

        # Список для YOLO-формата bounding boxes
        yolo_bboxes = []

        # Определяем случайное количество дефектов (от 1 до 6)
        num_defects = random.randint(1, N)

        for _ in range(num_defects):
            # Случайно выбираем дефект
            defect_path, defect_list, label = random.choice(defects_list)
            


            defect_name = random.choice(defect_list)
            defect_img = cv2.imread(os.path.join(defect_path, defect_name))


            
            # Задаем радиус дефекта
            radius = int(laptop_img.shape[0] / 60) + random.randint(-3, 3)

            if radius <= 5:
                radius = 5
            
            # Накладываем дефект и получаем bbox в YOLO-формате
            result, bbox = apply_defect(laptop_img, defect_img, mask_resized, radius=radius)
            
            # bbox в формате YOLO: [x_center, y_center, width, height]
            x_center, y_center, bbox_width, bbox_height = bbox
            
            img_h, img_w = laptop_img.shape[:2]
            
            # Преобразуем координаты из нормализованного YOLO-формата в пиксели
            x_min = int((x_center - bbox_width / 2) * img_w) - 2
            y_min = int((y_center - bbox_height / 2) * img_h) - 2
            x_max = int((x_center + bbox_width / 2) * img_w) + 2
            y_max = int((y_center + bbox_height / 2) * img_h) + 2
            
            # Вычитаем bbox из маски
            mask_resized[y_min:y_max, x_min:x_max] = 0  # Обновляем маску, вычитая область дефекта
            

            # Сохраняем YOLO bbox
            yolo_bboxes.append([label, x_center, y_center, bbox_width, bbox_height])

        # print(image_name, image_name.split('.')[0])

        # Сохраняем изображение с дефектами и обведенными bbox
        output_image_path = os.path.join('res\\output\\imgs', f'defect_{i}_{image_name}')
        cv2.imwrite(output_image_path, result)

        # Сохраняем YOLO bbox в текстовый файл
        img_labels = image_name.split('.')[0]
        output_bbox_path = os.path.join('res\\output\\labels', f'defect_{i}_{img_labels}.txt')
        with open(output_bbox_path, 'w') as f:
            for bbox in yolo_bboxes:
                f.write(f"{bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]} {bbox[4]}\n")

In [131]:
image_folder = 'res\\images\\monitor'
output_folder = 'res\\output'

In [137]:
def draw_dead_pixel(laptop_img, mask_resized):
    # Получаем координаты всех белых пикселей (где маска равна 255)
    screen_pixels = np.where(mask_resized == 255)
    
    if len(screen_pixels[0]) == 0 or len(screen_pixels[1]) == 0:
        print("Маска не содержит области для дефекта.")
        return laptop_img, None
    
    # Выбираем случайные координаты внутри маски
    idx = random.randint(0, len(screen_pixels[0]) - 1)
    y, x = screen_pixels[0][idx], screen_pixels[1][idx]

    # Случайный размер пикселя (например, от 2x2 до 10x10)
    pixel_size = random.randint(2, 10)

    # Случайный цвет пикселя
    pixel_color = [random.randint(0, 255) for _ in range(3)]  # RGB
    
    # Рисуем пиксель на изображении
    top_left = (x - pixel_size // 2, y - pixel_size // 2)
    bottom_right = (x + pixel_size // 2, y + pixel_size // 2)
    cv2.rectangle(laptop_img, top_left, bottom_right, pixel_color, -1)

    # Вычисляем bounding box в формате YOLO
    img_h, img_w = laptop_img.shape[:2]
    
    # Центр bounding box
    center_x = (top_left[0] + bottom_right[0]) / 2
    center_y = (top_left[1] + bottom_right[1]) / 2
    
    # Ширина и высота bounding box
    bbox_width = bottom_right[0] - top_left[0]
    bbox_height = bottom_right[1] - top_left[1]

    # YOLO формат: (x_center, y_center, width, height) в относительных координатах (0-1)
    bbox = [
        center_x / img_w,
        center_y / img_h,
        bbox_width / img_w,
        bbox_height / img_h
    ]

    return laptop_img, bbox

In [145]:
from aug import draw_dead_pixel

ImportError: cannot import name 'draw_dead_pixel' from 'aug' (c:\Jup\Новая папка\project\yolo_v11_test\model\aug.py)

In [143]:

laptop_img = cv2.imread('res\\images\\monitor\\57.jpg')
mask_img = cv2.imread('res\\images\\monitor\\masks\\MASK.57.jpg', 0)  # Маска (ч/б изображение), указывающая области для дефектов
mask_resized = cv2.resize(mask_img, (laptop_img.shape[1], laptop_img.shape[0]))
# Применение дефекта
result, bbox = draw_dead_pixel(laptop_img, mask_resized)
print (bbox)
# Сохранение результата
cv2.imwrite('res/laptop_with_defect.jpg', result)

[0.4075, 0.3075, 0.00125, 0.0016666666666666668]


True

In [135]:
for image_name in os.listdir(image_folder):
    # Пропускаем файлы, которые не являются изображениями
    if not image_name.lower().endswith(('.jpg', '.jpeg', '.webp')):
        continue
    mask_name = os.path.join(image_folder, 'masks')
    mask_name = os.path.join(mask_name, f'MASK.{image_name}')


    # Загружаем изображение
    image_path = os.path.join(image_folder, image_name)
    laptop_img = cv2.imread(image_path)
    print('---')
    if laptop_img is not None:
        print(image_path)



    mask_img = cv2.imread(mask_name, 0) 
    mask_resized = cv2.resize(mask_img, (laptop_img.shape[1], laptop_img.shape[0]))

    if mask_img is not None:
        print(mask_name)
    

   

---
res\images\monitor\57.jpg
res\images\monitor\masks\MASK.57.jpg
---
res\images\monitor\6443441131.jpg
res\images\monitor\masks\MASK.6443441131.jpg
---
res\images\monitor\6573311632.jpg
res\images\monitor\masks\MASK.6573311632.jpg
---
res\images\monitor\8XlofP_LzRb-77SSDwv9QHYCGwWfLhRCIvqhxWfRTnVhYqruuT47lLXZU2nWk5gdoS5owPNa8IAvprmJr7QHxmUp_GyoWg0Xu_12AQmI-UUlBUvAz6hVQ_S5tJ792VJ87ignZebiJgehjMb8UBYJGC6hLZXxQvtXsfl3yb3qUZC0.webp
res\images\monitor\masks\MASK.8XlofP_LzRb-77SSDwv9QHYCGwWfLhRCIvqhxWfRTnVhYqruuT47lLXZU2nWk5gdoS5owPNa8IAvprmJr7QHxmUp_GyoWg0Xu_12AQmI-UUlBUvAz6hVQ_S5tJ792VJ87ignZebiJgehjMb8UBYJGC6hLZXxQvtXsfl3yb3qUZC0.webp
---
res\images\monitor\akrales_180611_2654_0007.jpg
res\images\monitor\masks\MASK.akrales_180611_2654_0007.jpg
---
res\images\monitor\d72134a9d1bbe602f111ee82880c6756.jpeg
res\images\monitor\masks\MASK.d72134a9d1bbe602f111ee82880c6756.jpeg
---
res\images\monitor\i.webp
res\images\monitor\masks\MASK.i.webp
---
res\images\monitor\maxresdefault (1).jpg
res\im